__Run basic contamination screening and fastqc. perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/


In [ ]:
#make a list of read2 filenames to use for next loops
!ls *R2.fastq.gz > filenames
!sed -i'' -e 's/.fastq.gz//g' filenames #remove the extension

In [ ]:
#for some reason, barcodes still in at the start of read2, so use cutadapt to remove the first 6 reads from the read2 files. run in the python2.7 environment
for i in $(cat filenames_tofinish); do cutadapt -u 6 -o $i.cut.fastq.gz $i.fastq.gz; done

In [ ]:
#screen for contamination and filter out tagged reads simultaneously, filter 1 means keep only reads that mapped exclusively to the reference for A. percula. since this is already done for the read 1 files, just need to do it for the read 2 files that I cut the barcode from  
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*.cut.fastq.gz; do fastq_screen --tag --filter 1 --threads 32 --outdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/ $i; done

Using fastq_screen v0.13.0
Reading configuration from '/local/home/katrinac/miniconda3/envs/commandline/share/fastq-screen-0.13.0-0/fastq_screen.conf'
Aligner (--aligner) not specified, but BWA path and index files found: mapping with BWA
Using '/local/home/katrinac/miniconda3/envs/commandline/bin/bwa' as BWA path
Adding database Amphiprionpercula
Using 32 threads for searches
Option --subset set to 0: processing all reads in FASTQ files
Processing CAP10_APPC_34.05_R2.cut.fastq.gz
Not making data subset
Searching CAP10_APPC_34.05_R2.cut.fastq.gz_temp_subset.fastq against Amphiprionpercula
Filtering CAP10_APPC_34.05_R2.cut.tagged.fastq.gz
Filtering files with filter '1'
Filtering CAP10_APPC_34.05_R2.cut.tagged.fastq.gz
Processing complete

gzip: stdout: Broken pipe
Using fastq_screen v0.13.0
Reading configuration from '/local/home/katrinac/miniconda3/envs/commandline/share/fastq-screen-0.13.0-0/fastq_screen.conf'
Aligner (--aligner) not specified, but BWA path and index files found: map

Ignoring reads shorter than 20bp
Not making data subset
Searching CAP11_APPC_37.10_R2.cut.fastq.gz_temp_subset.fastq against Amphiprionpercula
Filtering CAP11_APPC_37.10_R2.cut.tagged.fastq.gz
Filtering files with filter '1'
Filtering CAP11_APPC_37.10_R2.cut.tagged.fastq.gz
Processing complete

gzip: stdout: Broken pipe
Using fastq_screen v0.13.0
Reading configuration from '/local/home/katrinac/miniconda3/envs/commandline/share/fastq-screen-0.13.0-0/fastq_screen.conf'
Aligner (--aligner) not specified, but BWA path and index files found: mapping with BWA
Using '/local/home/katrinac/miniconda3/envs/commandline/bin/bwa' as BWA path
Adding database Amphiprionpercula
Using 32 threads for searches
Option --subset set to 0: processing all reads in FASTQ files
Processing CAP11_APPC_37.12_R2.cut.fastq.gz
Ignoring reads shorter than 20bp
Not making data subset
Searching CAP11_APPC_37.12_R2.cut.fastq.gz_temp_subset.fastq against Amphiprionpercula
Filtering CAP11_APPC_37.12_R2.cut.tagged.fastq.gz

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 32 *tagged_filter.fastq.gz  $i; done

In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [ ]:
#make bwa index for reference fasta to use in mapping
!mkdir mapping
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping
!bwa index /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta

In [ ]:
#map using default parameters but add -M for picard compatibility, make sure to use the tagged/filtered
!for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/filenames); do bwa mem -t 60 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/fastq_screen/mapping/reference.fasta $i.R1.fastq.gz $i.R2.fastq.gz  2> bwa.$i.log | samtools sort -@$NUMProc -o $i-RG.bam 2>$i.bam.log; done  

In [ ]:
#make sure to check before running this, can't remember where .sai files are made...
#join pairs, convert from sam to bam files in screen -S samsort
#screen -S samsort
for i in $(cat APPC_sequencing/ultraplex_out/fastq_screen/filenames);
do bwa sampe /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping/reference.fasta $i\.read1.sai $i\.read2.sai $i\.R1.tagged_filter.fastq.gz $i\.R2.tagged_filter.fastq.gz | samtools view -bS - >  ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.bam; done



In [ ]:
#remove duplicate reads with picard
java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-R2.tagged_filter.fastq.gz O={}-RGmd.bam M={}_dup_metrics.txt OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log   

In [ ]:
#January 24, 2022 using fastp to trim adapters because trimmomatic left the barcode in read 2 file
for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames); 
    do fastp -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R1.fq.gz -I /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R2.fq.gz -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R1.fastp.fq.gz -O /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R2.fastp.fq.gz; 
    done



In [ ]:
#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/

for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames); 
    do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/APPCBarcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out; 
    done


In [ ]:
#this worked! do this for all capture pools, rename, move this line to the demultiplex notebook, and then redo the fastq, mapping, picard, then angsd and genotype calling
ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/Cap1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out 